In [1]:
# Cell 1: Import required libraries
import asyncio
import json
from mcp.client.stdio import stdio_client, StdioServerParameters
from mcp.client.session import ClientSession

print("✓ Libraries imported successfully")

✓ Libraries imported successfully


In [ ]:
# Cell 4: Add a raster layer (wetlands)
async def add_raster_layer(tiles):
    """Add a raster layer showing global wetlands"""
    server_params = StdioServerParameters(
        command="python",
        args=["server_sse.py"]
    )
    
    async with stdio_client(server_params) as (read, write):
        async with ClientSession(read, write) as session:
            await session.initialize()
            
            print("🌊 Adding wetlands layer...")
            result = await session.call_tool("add_layer", {
                "session_id": SESSION_ID,
                "id": "wetlands",
                "type": "raster",
                "source": {
                    "type": "raster",
                    "tiles": [tiles],
                    "tileSize": 256,
                    "attribution": "© OpenStreetMap contributors",
                    "minzoom": 0,
                    "maxzoom": 19
                },
                "visible": True
            })
            
            response = json.loads(result.content[0].text)
            if response.get('success'):
                print(f"✓ {response.get('message')}")
            else:
                print(f"✗ Error: {response.get('error')}")
            
            return response

# Run the function
result = await add_wetlands_layer()
print(f"\nFull response: {json.dumps(result, indent=2)}")

🌊 Adding wetlands layer...
✓ Added layer 'wetlands' to session notebook-test-session

Full response: {
  "success": true,
  "message": "Added layer 'wetlands' to session notebook-test-session",
  "layer": {
    "id": "wetlands",
    "type": "raster",
    "visible": true,
    "source": {
      "type": "raster",
      "tiles": [
        "https://tile.openstreetmap.org/{z}/{x}/{y}.png"
      ],
      "tileSize": 256,
      "attribution": "© OpenStreetMap contributors",
      "minzoom": 0,
      "maxzoom": 19
    },
    "layers": [
      {
        "id": "wetlands",
        "type": "raster",
        "source": "wetlands"
      }
    ],
    "paint": {},
    "filter": null
  }
}


In [10]:
# Cell 4a: Add OpenStreetMap basemap (WORKING TEST)
async def add_osm_basemap():
    """Add OpenStreetMap raster basemap - this should actually work!"""
    server_params = StdioServerParameters(
        command="python",
        args=["server_sse.py"]
    )
    
    async with stdio_client(server_params) as (read, write):
        async with ClientSession(read, write) as session:
            await session.initialize()
            
            print("🗺️ Adding OpenStreetMap basemap...")
            result = await session.call_tool("add_layer", {
                "session_id": SESSION_ID,
                "id": "osm-basemap",
                "type": "raster",
                "source": {
                    "type": "raster",
                    "tiles": ["https://tile.openstreetmap.org/{z}/{x}/{y}.png"],
                    "tileSize": 256,
                    "attribution": "© OpenStreetMap contributors"
                },
                "visible": True
            })
            
            response = json.loads(result.content[0].text)
            if response.get('success'):
                print(f"✓ {response.get('message')}")
            else:
                print(f"✗ Error: {response.get('error')}")
            
            return response

# Run the function
result = await add_osm_basemap()
print(f"\nFull response: {json.dumps(result, indent=2)}")

🗺️ Adding OpenStreetMap basemap...
✓ Added layer 'osm-basemap' to session notebook-test-session

Full response: {
  "success": true,
  "message": "Added layer 'osm-basemap' to session notebook-test-session",
  "layer": {
    "id": "osm-basemap",
    "type": "raster",
    "visible": true,
    "source": {
      "type": "raster",
      "tiles": [
        "https://tile.openstreetmap.org/{z}/{x}/{y}.png"
      ],
      "tileSize": 256,
      "attribution": "\u00a9 OpenStreetMap contributors"
    },
    "layers": [
      {
        "id": "osm-basemap",
        "type": "raster",
        "source": "osm-basemap"
      }
    ],
    "paint": {},
    "filter": null
  }
}


In [ ]:
# Cell 5: Add a vector layer (protected areas)
async def add_protected_areas():
    """Add a vector layer showing WDPA protected areas"""
    server_params = StdioServerParameters(
        command="python",
        args=["server_sse.py"]
    )
    
    async with stdio_client(server_params) as (read, write):
        async with ClientSession(read, write) as session:
            await session.initialize()
            
            print("🏞️  Adding WDPA protected areas layer...")
            result = await session.call_tool("add_layer", {
                "session_id": SESSION_ID,
                "id": "wdpa",
                "type": "vector",
                "source": {
                    "type": "vector",
                    "tiles": ["https://demotiles.maplibre.org/tiles/{z}/{x}/{y}.pbf"],
                    "minzoom": 0,
                    "maxzoom": 5
                },
                "layers": [
                    {
                        "id": "wdpa-fill",
                        "type": "fill",
                        "source": "wdpa",
                        "source-layer": "countries",
                        "paint": {
                            "fill-color": "#00ff00",
                            "fill-opacity": 0.3
                        }
                    },
                    {
                        "id": "wdpa-outline",
                        "type": "line",
                        "source": "wdpa",
                        "source-layer": "countries",
                        "paint": {
                            "line-color": "#006600",
                            "line-width": 1
                        }
                    }
                ],
                "visible": True
            })
            
            response = json.loads(result.content[0].text)
            if response.get('success'):
                print(f"✓ {response.get('message')}")
            else:
                print(f"✗ Error: {response.get('error')}")
            
            return response

# Run the function
result = await add_protected_areas()
print(f"\nFull response: {json.dumps(result, indent=2)}")

In [ ]:
# Cell 6: Filter protected areas to show only IUCN Category II (National Parks)
async def filter_to_national_parks():
    """Filter WDPA to show only IUCN Category II (National Parks)"""
    server_params = StdioServerParameters(
        command="python",
        args=["server_sse.py"]
    )
    
    async with stdio_client(server_params) as (read, write):
        async with ClientSession(read, write) as session:
            await session.initialize()
            
            print("🔍 Filtering to show only National Parks (IUCN Category II)...")
            result = await session.call_tool("filter_layer", {
                "session_id": SESSION_ID,
                "layer_id": "wdpa-fill",
                "filter": ["==", "IUCN_CAT", "II"]
            })
            
            response = json.loads(result.content[0].text)
            if response.get('success'):
                print(f"✓ {response.get('message')}")
            else:
                print(f"✗ Error: {response.get('error')}")
            
            return response

# Run the function
result = await filter_to_national_parks()
print(f"\nFull response: {json.dumps(result, indent=2)}")

In [9]:
# Cell 7: Set map view to focus on the United States
async def set_us_view():
    """Set map view to show the United States"""
    server_params = StdioServerParameters(
        command="python",
        args=["server_sse.py"]
    )
    
    async with stdio_client(server_params) as (read, write):
        async with ClientSession(read, write) as session:
            await session.initialize()
            
            print("🗺️  Setting map view to United States...")
            result = await session.call_tool("set_map_view", {
                "session_id": SESSION_ID,
                "center": [-98.5795, 39.8283],  # Center of US
                "zoom": 4
            })
            
            response = json.loads(result.content[0].text)
            if response.get('success'):
                print(f"✓ {response.get('message')}")
            else:
                print(f"✗ Error: {response.get('error')}")
            
            return response

# Run the function
result = await set_us_view()
print(f"\nFull response: {json.dumps(result, indent=2)}")

🗺️  Setting map view to United States...
✓ None

Full response: {
  "success": true,
  "center": [
    -98.5795,
    39.8283
  ],
  "zoom": 4
}


In [ ]:
# Cell 8: List all layers in our session
async def list_session_layers():
    """List all layers currently in our session"""
    server_params = StdioServerParameters(
        command="python",
        args=["server_sse.py"]
    )
    
    async with stdio_client(server_params) as (read, write):
        async with ClientSession(read, write) as session:
            await session.initialize()
            
            print("📋 Listing all layers in session...")
            result = await session.call_tool("list_layers", {
                "session_id": SESSION_ID
            })
            
            response = json.loads(result.content[0].text)
            if response.get('success'):
                layers = response.get('layers', [])
                print(f"\n✓ Found {len(layers)} layer(s):\n")
                for layer in layers:
                    visible = "✓" if layer.get('visible') else "✗"
                    print(f"  {visible} {layer['id']} ({layer['type']})")
            else:
                print(f"✗ Error: {response.get('error')}")
            
            return response

# Run the function
result = await list_session_layers()
print(f"\nFull response: {json.dumps(result, indent=2)}")

## Viewing Your Map

To see the map with your changes:

1. Make sure the MCP server is running in the background or start it:
   ```bash
   python server_sse.py
   ```

2. Open your browser to: http://localhost:8081

3. Set the session cookie to match this notebook's session:
   - Open DevTools (F12)
   - Go to Console
   - Run: `document.cookie = 'mcp_map_session=notebook-test-session; path=/'; location.reload();`

4. Your map should now show:
   - Wetlands layer (raster)
   - WDPA protected areas filtered to IUCN Category II (National Parks)
   - Centered on the United States

## Next Steps

Try modifying the cells above to:

- Add different layers from examples.md
- Change the styling (colors, opacity)
- Apply different filters
- Change the map view to other locations
- Toggle layer visibility

See the [examples.md](examples.md) file for more ideas!